In [1]:
import pandas as pd
import numpy as np
train_data=pd.read_csv(r"C:\Users\Abhi9\OneDrive\Documents\.vscode\c progrmming\Python\MachineLearning\WOC\nn_train.csv")
test_data=pd.read_csv(r"C:\Users\Abhi9\OneDrive\Documents\.vscode\c progrmming\Python\MachineLearning\WOC\nn_test.csv")
train_data["class_label"].unique()

array([ 5,  9,  8, 10,  3,  1,  4,  6,  2,  7])

In [18]:
dropped_train=train_data.drop(['ID'],axis=1)
train_data_class_label=dropped_train.drop(['binary_label'],axis=1)
x_train=train_data_class_label.iloc[:,:-1].values
row=x_train.shape[0]
y_train=train_data_class_label.iloc[:,-1].values.reshape(1,row)
x_test=test_data.drop(['ID'],axis=1).values
print(x_train.shape,y_train.shape)
test_data.shape

(80000, 1024) (1, 80000)


(20000, 1025)

In [3]:
def relu(z):
    return np.maximum(0,z)

In [4]:
def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # Stability trick
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

In [5]:
def backward_propagation(X, Y_one_hot,Z1,A1,Z2,A2, W1, W2):
    m = X.shape[1]

    dZ2 = A2 - Y_one_hot
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (Z1 > 0)  # Derivative of ReLU
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    return dW1, db1, dW2, db2


In [6]:
def predict(x_, w1, b1, w2, b2):
    z1=w1@x_+b1
    a1=relu(z1)
    z2=w2@a1 + b2
    a2=softmax(z2)
    predictions = np.argmax(a2, axis=0) + 1  # Add 1 to match class labels (1-10)
    return predictions


In [12]:
epochs=300
hidden_layer=5
n,m=x_train.T.shape
classes=10
learning_rate=0.01
w1=np.random.randn(hidden_layer,n)*0.01
b1=np.zeros((hidden_layer,1))*0.01
w2=np.random.randn(classes+1,hidden_layer)*0.01
b2=np.zeros((classes+1,1))
y_one_hot=np.eye(classes+1)[y_train.T.reshape(-1)].T


In [13]:
for i in range(epochs):
    z1=w1@x_train.T+b1
    a1=relu(z1)
    z2=w2@a1 + b2
    a2=softmax(z2)
    loss = -np.sum(y_one_hot * np.log(a2 + 1e-9)) / m
    dW1, db1, dW2, db2 = backward_propagation(x_train.T, y_one_hot, z1,a1,z2,a2, w1, w2)
    w1-=learning_rate*dW1
    w2-=learning_rate*dW2
    b1-=learning_rate*db1
    b2-=learning_rate*db2
    if i % 10 == 0:
        print(f"Epoch {i}, Loss: {loss}")
    

Epoch 0, Loss: 2.399828599858363
Epoch 10, Loss: 2.3633484802099183
Epoch 20, Loss: 2.396330052574053
Epoch 30, Loss: 2.39545299561284
Epoch 40, Loss: 2.3945934778268785
Epoch 50, Loss: 2.3937491236588824
Epoch 60, Loss: 2.392919675607183
Epoch 70, Loss: 2.39210453276131
Epoch 80, Loss: 2.391303202215943
Epoch 90, Loss: 2.3905153444827065
Epoch 100, Loss: 2.3897407355005984
Epoch 110, Loss: 2.388978883071103
Epoch 120, Loss: 2.388229282676752
Epoch 130, Loss: 2.387491051907748
Epoch 140, Loss: 2.3867644237063916
Epoch 150, Loss: 2.386048667530919
Epoch 160, Loss: 2.3853414688947354
Epoch 170, Loss: 2.3846369222133648
Epoch 180, Loss: 2.3838925382298988
Epoch 190, Loss: 2.3833284323708024
Epoch 200, Loss: 2.3826709762633027
Epoch 210, Loss: 2.3820237177210415
Epoch 220, Loss: 2.3813864273990073
Epoch 230, Loss: 2.3807588780312163
Epoch 240, Loss: 2.3801408484283235
Epoch 250, Loss: 2.3795321379387238
Epoch 260, Loss: 2.3789325424811865
Epoch 270, Loss: 2.3783418636805744
Epoch 280, Loss

In [14]:
prediction=predict(x_test.T,w1,b1,w2,b2)
    

In [34]:
predicted_class_label=predict(x_test.T,w1,b1,w2,b2)
predicted_class_label_=pd.DataFrame(predicted_class_label.reshape((20000,1)))
predicted_class_label_=predicted_class_label.reshape(20000,1)
predicted_class_label__=pd.DataFrame(predicted_class_label_)

In [35]:
predicted_binary_label_through_Neural_Network=test_data.join(predicted_class_label__)
predicted_binary_label_through_Neural_Network.shape

(20000, 1026)

In [36]:

predicted_binary_label_through_Neural_Network.to_csv('predicted_class_label_through_Neural_Network.csv')